# Alpha-Beta seperately

Our previous model trained on the entire dataset (with both alpha and beta). Now we will train two models, one for alpha and one for beta. This will give us two prediction, which we will then average to get the final prediction.

In [1]:
# andere baseline? Beta en alpha keten nu. Als baseline: 2 modellen: eentje voor beta, eentje voor alfa. Beta trainen op alpha keten, alfa
# baseline beter werken dan clf met enkel alfa of beta? of model trainen op

# nieuwe preprint: nieuwe benchmark dataset. Pieter zet op Slack.

In [2]:
from util import get_train_dataset, get_features, fix_test, plot_roc_curve
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split

df = get_train_dataset()
# df = df.sample(n=1000) # for faster debugging
train, test = train_test_split(df, test_size=0.2, random_state=42)

x = get_features(train)  # contains both alfa and beta features
y = train['reaction']

# Keep only the columns starting with 'alpha_'
x_alpha = x[x.columns[x.columns.str.startswith('alfa_')]]
x_beta = x[x.columns[x.columns.str.startswith('beta_')]]

x_test = get_features(test, test=True) # Note: Without the test=True, I'm still getting the almost perfect prediciton error
x_test  = fix_test(x_test, x.columns)
y_test = test['reaction']

x_test_alpha = x_test[x_test.columns[x_test.columns.str.startswith('alfa_')]]
x_test_beta = x_test[x_test.columns[x_test.columns.str.startswith('beta_')]]

KeyError: "['TRBV', 'TRBJ'] not in index"

Try a random forest classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Random forest doesn't support NaN values
x_alpha = x_alpha.fillna(0)
x_beta = x_beta.fillna(0)
x_test_alpha = x_test_alpha.fillna(0)
x_test_beta = x_test_beta.fillna(0)

clf_alpha = RandomForestClassifier(n_estimators=200, random_state=42)
clf_beta = RandomForestClassifier(n_estimators=200, random_state=42)

In [ ]:
clf_alpha.fit(x_alpha, y)
clf_beta.fit(x_beta, y)

In [ ]:
y_pred_alpha = clf_alpha.predict_proba(x_test_alpha)[:, 1]
y_pred_beta = clf_beta.predict_proba(x_test_beta)[:, 1]

In [ ]:
from sklearn import metrics

def calculate_auc_and_plot(y_test, y_pred):

    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=1)
    roc_auc = metrics.auc(fpr, tpr)

    plot_roc_curve(fpr, tpr, label=f'ROC curve (area = {roc_auc:.3f})', title='ROC curve')

In [ ]:
# Calculate AUC for alpha
calculate_auc_and_plot(y_test, y_pred_alpha) # TODO: kijken of veel alpha's zelfde score?

In [ ]:
# Calculate AUC for beta
calculate_auc_and_plot(y_test, y_pred_beta)

In [ ]:
y_pred = (y_pred_alpha + y_pred_beta) / 2

In [ ]:
calculate_auc_and_plot(y_test, y_pred)

In [ ]:
# y_pred is predicted using predict_proba, so we need to convert it to 0/1
y_pred_label = (y_pred > 0.5).astype(int)
print(metrics.classification_report(y_test, y_pred_label))
print(metrics.confusion_matrix(y_test, y_pred_label))

In [ ]:
# Instead of averaging the predictions, we can also take the maximum or minimum
import numpy as np
y_pred_max = np.maximum(y_pred_alpha, y_pred_beta)

calculate_auc_and_plot(y_test, y_pred_max)


In [ ]:
y_pred_max_label = (y_pred_max > 0.5).astype(int)
print(metrics.classification_report(y_test, y_pred_max_label))
print(metrics.confusion_matrix(y_test, y_pred_max_label))

In [ ]:
# Or the minimum
y_pred_min = np.minimum(y_pred_alpha, y_pred_beta)

calculate_auc_and_plot(y_test, y_pred_min)

In [ ]:
y_pred_min_label = (y_pred_min > 0.5).astype(int)
print(metrics.classification_report(y_test, y_pred_min_label))
print(metrics.confusion_matrix(y_test, y_pred_min_label))

In [ ]:
# Or the product
y_pred_product = y_pred_alpha * y_pred_beta

calculate_auc_and_plot(y_test, y_pred_product)

In [ ]:
y_pred_product_label = (y_pred_product > 0.5).astype(int)
print(metrics.classification_report(y_test, y_pred_product_label))
print(metrics.confusion_matrix(y_test, y_pred_product_label))

Product and average seem to perform the best, yielding a similar result as the model trained on the entire dataset. (and better than the model trained on only alpha or beta).

# Complete only
I'm interested now in how the missing values affects those predictions. Let's see what happens if we remove the rows with missing values.

In [ ]:
from util import get_train_dataset, get_features, fix_test, plot_roc_curve
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split

df = get_train_dataset()
# df = df.sample(n=1000) # for faster debugging

df_length = len(df)
df = df.dropna() # Must be here and not after the get_features, since get_features might also introduce NaN values
print(f'Number of rows removed: {df_length - len(df)} ({(df_length - len(df)) / df_length * 100:.2f}%)')

train, test = train_test_split(df, test_size=0.2, random_state=42)

x = get_features(train)  # contains both alfa and beta features
y = train['reaction']

# Keep only the columns starting with 'alpha_'
x_alpha = x[x.columns[x.columns.str.startswith('alfa_')]]
x_beta = x[x.columns[x.columns.str.startswith('beta_')]]

x_test = get_features(test, test=True) # Note: Without the test=True, I'm still getting the almost perfect prediciton error
x_test  = fix_test(x_test, x.columns)
y_test = test['reaction']

x_test_alpha = x_test[x_test.columns[x_test.columns.str.startswith('alfa_')]]
x_test_beta = x_test[x_test.columns[x_test.columns.str.startswith('beta_')]]

# dit model hierboven ook test op zelfde testset
# is dat omdat model beter is of omdat testset makkelijker te voorspellen?
# literatuur induiken, algemeen missing data

In [ ]:
# Remove NaN's introduced by get_features
x_alpha = x_alpha.fillna(0)
x_beta = x_beta.fillna(0)
x_test_alpha = x_test_alpha.fillna(0)
x_test_beta = x_test_beta.fillna(0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf_alpha = RandomForestClassifier(n_estimators=200, random_state=42)
clf_beta = RandomForestClassifier(n_estimators=200, random_state=42)

In [ ]:
clf_alpha.fit(x_alpha, y)
clf_beta.fit(x_beta, y)

In [ ]:
y_pred_alpha = clf_alpha.predict_proba(x_test_alpha)[:, 1]
y_pred_beta = clf_beta.predict_proba(x_test_beta)[:, 1]

In [ ]:
calculate_auc_and_plot(y_test, y_pred_alpha)

In [ ]:
calculate_auc_and_plot(y_test, y_pred_beta)

In [ ]:
y_pred = (y_pred_alpha + y_pred_beta) / 2

In [ ]:
calculate_auc_and_plot(y_test, y_pred)

In [ ]:
y_pred_label = (y_pred > 0.5).astype(int)
print(metrics.classification_report(y_test, y_pred_label))
print(metrics.confusion_matrix(y_test, y_pred_label))

Alpha tells us more than beta here.

In [ ]:
# Let's also create one model using both alpha and beta features
from util import get_train_dataset, get_features, fix_test, plot_roc_curve # calculate_auc_and_plot
from sklearn.model_selection import train_test_split


df = get_train_dataset()
# df = df.sample(n=1000) # for faster debugging
df = df.dropna()

train, test = train_test_split(df, test_size=0.2, random_state=42)

x = get_features(train)  # contains both alfa and beta features
y = train['reaction']

x_test = get_features(test, test=True)
x_test  = fix_test(x_test, x.columns)
y_test = test['reaction']

# Remove NaN's introduced by get_features
x = x.fillna(0)
x_test = x_test.fillna(0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(x, y)

In [ ]:
from util import calculate_auc_and_plot
y_pred = clf.predict_proba(x_test)[:, 1]
calculate_auc_and_plot(y_test, y_pred)

# Summary
## Seperate models
I tried generating a seperate model for the alpha part and a seperate model for the beta part. Both models used a random forest classifier (since this has been one of the best models for this) and the NaN's where filled with 0. I didn't drop any rows, so the alpha model contained a lot of rows containing only zeros.

The alpha model was trained on all alpha columns, the beta model on all beta columns. Those classifiers give a probability as output. The ROC curve of the alpha model contained a straight line, which is probably because of all 0 rows. Alpha model had an AUC of 0.767, the beta model 0.794.

I tried different ways of combining the results of those models. Taking the average resulted in an ROC of 0.876 (which is the same as when using one random forest model on all columns). The misclassifications where mainly false positivess. Product gave similar results. Maximum and minimum slightly worse.

## Complete only
I was also interested in the accuracy if I dropped all rows conaining NaN's (58% of the rows). I created a seperate model for the alpha columns and a seperate model for the beta columns. This resulted in an AUC of 0.981 for the alpha model, 0.887 for the beta model. The combination (avg) had an AUC of 0.983, which is slightly higher than one model trained on both alpha and beta together (0.968).

Model met op nan's getraind, zien hoe dat werkt op de testsetmet nan's (wel meer data om op te trainen), vooral vraag is er een meerwaarde om nan's mee te rekenen om modellen beter te maken'

# NaN's in train, not in test
A seperate test to see whether the NaN's actually improved the model (even if the test set doesn't contain any NaN's).

In [ ]:
from util import get_train_dataset, get_features, fix_test, plot_roc_curve
from sklearn.model_selection import train_test_split

df = get_train_dataset()
# df = df.sample(n=1000) # for faster debugging

In [ ]:
train, test = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
df_test_length = len(test)
test = test.dropna()
print(f'Number of rows removed: {df_test_length - len(test)} ({(df_test_length - len(test)) / df_test_length * 100:.2f}%)')

In [ ]:
x = get_features(train)
y = train['reaction']

## Alpha and beta together

In [ ]:
x_test = get_features(test, test=True) # Note: Without the test=True, I'm still getting the almost perfect prediciton error
x_test  = fix_test(x_test, x.columns)
y_test = test['reaction']

In [ ]:
x = x.fillna(0)
x_test = x_test.fillna(0)

In [ ]:
clf = RandomForestClassifier(random_state=42)
clf.fit(x, y)

In [ ]:
y_pred = clf.predict_proba(x_test)[:, 1]
calculate_auc_and_plot(y_test, y_pred)

## Alpha and beta seperately

In [ ]:
# Keep only the columns starting with 'alpha_'
x_alpha = x[x.columns[x.columns.str.startswith('alfa_')]]
x_beta = x[x.columns[x.columns.str.startswith('beta_')]]

In [ ]:
x_test_alpha = x_test[x_test.columns[x_test.columns.str.startswith('alfa_')]]
x_test_beta = x_test[x_test.columns[x_test.columns.str.startswith('beta_')]]

In [ ]:
x_alpha = x_alpha.fillna(0)
x_beta = x_beta.fillna(0)
x_test_alpha = x_test_alpha.fillna(0)
x_test_beta = x_test_beta.fillna(0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf_alpha = RandomForestClassifier(n_estimators=200, random_state=42)
clf_alpha.fit(x_alpha, y)
y_pred_alpha = clf_alpha.predict_proba(x_test_alpha)[:, 1]
calculate_auc_and_plot(y_test, y_pred_alpha)

In [ ]:
clf_beta = RandomForestClassifier(n_estimators=200, random_state=42)
clf_beta.fit(x_beta, y)
y_pred_beta = clf_beta.predict_proba(x_test_beta)[:, 1]
calculate_auc_and_plot(y_test, y_pred_beta)

In [ ]:
y_pred = (y_pred_alpha + y_pred_beta) / 2
calculate_auc_and_plot(y_test, y_pred)

In [ ]:
from sklearn import metrics
y_pred_label = (y_pred > 0.5).astype(int)
print(metrics.classification_report(y_test, y_pred_label))
print(metrics.confusion_matrix(y_test, y_pred_label))